In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import matplotlib.pyplot as plt
import cv2
import numpy as np
import tensorflow as tf
import collections
import sys
import Structural_Perturbations as SP
tf.logging.set_verbosity(tf.logging.ERROR)
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
# os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

"""Specifies batch size, number of classes in data and number of epochs"""
batch_size = 128
num_classes = 10
epochs = 10

"""input image dimensions""" 
img_rows, img_cols = 28, 28

"""Load/Download Dataset"""
from keras.datasets import mnist
train_data,train_labels,eval_data,eval_labels = SP.load('mnist')

In [ ]:
"""Defines architecture of the model. We use a 2 conv-layer followed by a fully-connected layer with 
   max-pooling and dropout in between layers. Loss is categorical cross-entropy and optimiser used is Adam"""
def new_model():
    model = Sequential()
    model.add(
        Conv2D(
            32, kernel_size=(3, 3), activation='relu',
            input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(
        loss=keras.losses.categorical_crossentropy,
        optimizer=keras.optimizers.Adam(),
        metrics=['accuracy'])
    return model

"""Evaluates a given model over a range of perturbation
   Local Variables : 
   
   res : stores the perturbation and accuracy of the model on that pertubation
   model : the loaded model
   base : baseline accuracy of the model
   name : name of the perturbation
   alpha : the array of perturbation to check
   d : the original unperturbed dataset"""
def evaluate(model, base, d, name, alpha):
    global epsilon
    print("Number of uncovered points", alpha.shape)
    res = []
    for i in alpha:
        data = np.copy(d)
        data = SP.Transform(name, data, i)
        labels = data[:, -1]
        labels = keras.utils.to_categorical(labels, 10)
        data = data[:, 0:784]
        data = data.reshape(-1, 28, 28, 1)
        m1 = model.evaluate(
            data, labels, verbose=0, batch_size=2048)[1]
        res.append([i, m1])
    res = np.array(res)
    return res


"""Perturbs training dataset and trains a given model on a perturbed dataset. Global variable sym denotes
    symmetric/asymmetric training method. If sym=1, symmetric training, hence append symmetric points.
    Returns model"""
def train_model(name, d, alpha):
    global sym
    data = np.copy(d)
    if sym==1:
        if name == 'Perspective':
            alpha.append(56-alpha[-1])
        elif name == "Scaling":
            alpha.append(2-alpha[-1])
        else:
            alpha.append(-alpha[-1])
    print("To be trained on ", alpha)
    for i in alpha:
        if i != alpha[0]:
            perturbed_data = SP.Transform(name, d, i)
            data = np.concatenate((data, perturbed_data))
    model = new_model()
    labels = data[:, -1]
    labels = keras.utils.to_categorical(labels, 10)
    data = data[:, 0:784].reshape(-1, 28, 28, 1)
    model.fit(data, labels, batch_size=batch_size, epochs=epochs, verbose=1)
    return model

"""Loads data and stacks the labels as a column of data. This is required for multiprocessing."""
def load_data():
    train_data, train_labels, eval_data, eval_labels = SP.load('mnist')
    train_data = train_data.reshape(-1, 784)
    train_labels = train_labels.reshape(-1, 1)
    train = np.hstack((train_data, train_labels))
    eval_data = eval_data.reshape(-1, 784)
    eval_labels = eval_labels.reshape(-1, 1)
    input_shape = (img_rows, img_cols, 1)
    test = np.hstack((eval_data, eval_labels))
    return (train, test, input_shape)

"""Finds the centrepoint given an array of [perturbation,accuracy]"""
def centerpoint(res):
    res = np.array(res)
    res = res[np.lexsort(np.fliplr(res).T)]
    m = np.where(res[:,1] == min(res[:,1]))
    m = res[m]
    return m[0][0]

In [ ]:
"""Loads the data and Vanilla model"""
train,test,input_shape = load_data()
model = load_model("MNIST_vanilla.h5")

In [ ]:
"""Finds the baseline accuracy of the vanilla model.
   Variables:
   
   res : Stores the results array of accurcy vs perturbation
   
   sym : 1 denotes symmetric training, 0 asymmetric training
   
   points : Initialize with initial (natural) pertubation.
            Scaling : 1
            Perspective : 28
            Else : 0
            Stores the points on which derivative was maximum and hence needs training.
            
   epsilon : Defines our threshold. 0.1 = 10% accuracy drop
   
   max_perturbation : Stores the maximum allowed perturbation
   
   name : Name of pertubration {Exposure, Scaling, Rotation, Translation, Shear, Perspective}
   
   x : step_length
   
   set_to_test : the array of pertubation to test on(uncovered points). 
                 Scaling ranges from 0 to a positive number.
                 Perspective ranges from 56-max to 28+max
                 Others range from -max to +max"""

base = model.evaluate(train_data.reshape(-1,28,28,1),keras.utils.to_categorical(train_labels,10),verbose=0)[1]
res = []
points = []
epsilon = 0.1
print("Base Accuracy is ",base)

"""Make sure to change these three variables appropriately"""
name = "Rotation"
max_perturbation = 90
x = 10
sym = 1

set_to_test = []
if name == "Scaling":
    set_to_test = np.arange(2-max_perturbation,max_perturbation+x,x)
    points = [1]
elif name == "Perspective":
    set_to_test = np.arange(56-max_perturbation,max_perturbation+x,x)
    points = [28]
else:
    set_to_test = np.arange(-max_perturbation,max_perturbation+x,x)
    points = [0]
s = np.copy(set_to_test)

In [ ]:
"""The main loop. It continues til the model has >= accuracy than threshold on all pertubations in range"""

while(True):
    r = evaluate(model,base,train,name,set_to_test)
    if min(r[:,1]) < base-epsilon:
         i = centerpoint(r)
    else:
        break
    u = 0
    res.append(r)
    """Delete covered points"""
    while(u<r.shape[0]):
        if r[u][1] > (base - epsilon):
            r = np.delete(r,(u),axis=0)
        else:
            u+=1
    set_to_test = r[:,0]
    if i<=max_perturbation:
        points.append(i)
        print(points)
        model= train_model(name,train,points)
        print("New model created")
print("Model is robust")
points = np.array(points).round(decimals=2)
res = np.array(res)

In [ ]:
"""Test the robust model on test data over same pertubation range. Vertical lines show the training points"""
r = evaluate(model,base,test,name,s)
plt.xlabel(name,fontsize = 18)
plt.ylim(0,1.1)
for i in points:
    plt.axvline(i,c='red',linewidth=0.5)
plt.ylabel("Accuracy",fontsize = 18)
plt.plot(r[:,0],r[:,1])
plt.show()

In [ ]:
"""Save the model, results array and array of points"""
model.save("MNIST_" + name +".h5")
np.save("MNIST_" + name +"_res.npy",res)
np.save("MNIST_"+ name +"_points.npy",points)